In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import Sequential, Input, Model
from keras.layers import (Dense, Conv1D, Convolution2D, GRU, LSTM, Recurrent, Bidirectional, TimeDistributed,
                          Dropout, Flatten, RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
from keras.layers.core import Lambda
from keras.optimizers import Adam
from keras.regularizers import l1
from keras.models import load_model
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from 

Using TensorFlow backend.


In [3]:
def function_gen(x, a, b, c, d, e, f, g):
    y1 = a*(x-e)**3 + b*(x-f)**2+c*(x-g)+d
    y2 = d*(x-g)**2 + c*x +a
    return y1, y2

In [105]:
x = np.linspace(0,1,100)

y1 = []
y2 = []
a = []
b = []
c = []
d = []
e = np.random.uniform(0,10)
f = np.random.uniform(0,10)
g = np.random.uniform(0,10)
for i in range(1000):

    a_ = np.random.normal(8, 1.5, 1)
    b_ = np.random.normal(2, 2, 1)
    c_ = np.random.normal(4, 1, 1)
    d_ = np.random.normal(0, 0.5, 1)

    y1_, y2_ = function_gen(x, a_, b_, c_, d_, e, f, g)
    
    y1.append(y1_)
    y2.append(y2_)
    a.append(a_)
    b.append(b_)
    c.append(c_)
    d.append(d_)

In [13]:
# def rnn_auto(layer , size, num_encode_layers, num_decode_layers, embedding, n_step, lr = 3e-6, drop_frac=0.,bidirectional=False, l1_norm = 1e-4,**kwargs):
   """
    Function which builds the reccurrent neural network autoencoder
    Parameters
    ----------
    layer : string; options: 'lstm','gru'
        selects the layer type
    size  : numpy, int
        sets the size of encoding and decoding layers in the network
    num_encode_layers  : numpy, int
        sets the number of encoding layers in the network
    num_decode_layers : numpy, int
        sets the number of decoding layers in the network
    embedding : numpy, int
        sets the size of the embedding layer
    n_steps : numpy, int
        length of the input time series
    lr : numpy, float
        sets the learning rate for the model
    drop_frac : numpy, float
        sets the dropout fraction
    bidirectional : numpy, bool
        selects if the model is linear or bidirectional
    l1_norm : numpy. float
        sets the lambda value of the l1 normalization. The larger the value the greater the
        sparsity. None can be passed to exclude the use or l1 normailzation.
    Returns
    -------
    model : Keras, object
        Keras tensorflow model
    """
#     # defines the model
#     model = Sequential()

#     # selects if the model is bidirectional
#     if bidirectional:
#         wrapper = Bidirectional
#         # builds the first layer
#         model.add(Bidirectional(layer(size, return_sequences=(num_encode_layers > 1),  dropout=drop_frac),
#                             input_shape=(n_step, 1)))
#     else:
#         wrapper = lambda x: x
#         # builds the first layer
#         model.add(wrapper(layer(size, return_sequences=(num_encode_layers > 1),  dropout=drop_frac,
#                 input_shape=(n_step, 1))))

#     # builds the encoding layers
#     for i in range(1, num_encode_layers):
#         model.add(wrapper(layer(size, return_sequences=(i < num_encode_layers - 1), dropout=drop_frac)))

#     # builds the embedding layer
#     if l1_norm == None:
#         # embedding layer without l1 regulariization
#         model.add(Dense(embedding, activation='relu', name='encoding'))
#     else:
#         # embedding layer with l1 regularization
#         model.add(Dense(embedding, activation='relu', name='encoding',activity_regularizer=l1(l1_norm)))

#     # builds the repeat vector
#     model.add(RepeatVector(n_step))

#     # builds the decoding layer
#     for i in range(num_decode_layers):
#         model.add(wrapper(layer(size, return_sequences=True, dropout=drop_frac)))

#     # builds the time distributed layer to reconstruct the original input
#     model.add(TimeDistributed(Dense(1, activation='linear')))

#     # complies the model
#     model.compile(Adam(lr), loss='mse')

#     # returns the model
#     return model

# # def train_model(model, data_train, data_test, path, epochs, batch_size):

# #     #builds the filename
# #     filepath = path + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
# #     # sets the control of checkpoints
# #     checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True,
# #                                              save_weights_only=True, mode='min', period=1)

# #     # fits the model
# #     model.fit(np.atleast_3d(data1), np.atleast_3d(data2), epochs=250000,
# #           batch_size=1200, validation_data=(np.atleast_3d(data), np.atleast_3d(data)),
# #           callbacks=[tbCallBack, checkpoint])



SyntaxError: non-default argument follows default argument (<ipython-input-13-dfb353eb3ca4>, line 1)

In [149]:
def create_model( layer = LSTM, size = 8, num_encode_layers = 1 , num_decode_layers = 1, embedding = 16, n_step = 100, lr = 3e-6, drop_frac=0.,bidirectional=False, l1_norm = 1e-4, **kwargs):
    model = Sequential()
    if bidirectional:
        wrapper = Bidirectional
        # builds the first layer
        model.add(Bidirectional(layer(size, return_sequences=(num_encode_layers > 1),  dropout=drop_frac),
                            input_shape=(n_step, 1)))
    else:
        wrapper = lambda x: x
        # builds the first layer
        model.add(wrapper(layer(size, return_sequences=(num_encode_layers > 1),  dropout=drop_frac,
                input_shape=(n_step, 1))))
        
    for i in range(1, num_encode_layers):
        model.add(wrapper(layer(size, return_sequences=(i < num_encode_layers - 1), dropout=drop_frac)))

    # builds the embedding layer
    if l1_norm == None:
        # embedding layer without l1 regulariization
        model.add(Dense(embedding, activation='relu', name='encoding'))
    else:
        # embedding layer with l1 regularization
        model.add(Dense(embedding, activation='relu', name='encoding',activity_regularizer=l1(l1_norm)))

    # builds the repeat vector
    model.add(RepeatVector(n_step))

    # builds the decoding layer
    for i in range(num_decode_layers):
        model.add(wrapper(layer(size, return_sequences=True, dropout=drop_frac)))

    # builds the time distributed layer to reconstruct the original input
    model.add(TimeDistributed(Dense(1, activation='linear')))

    # complies the model
    model.compile(Adam(lr), loss='mse')

    # returns the model
    return model 

In [119]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

Epoch 1/100
100/100 [==============================] - 2s 23ms/step - loss: 253.4781
Epoch 2/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4698
Epoch 3/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4607
Epoch 4/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4514
Epoch 5/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4429
Epoch 6/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4349
Epoch 7/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4257
Epoch 8/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4174
Epoch 9/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4086
Epoch 10/100
100/100 [==============================] - 1s 8ms/step - loss: 253.4002
Epoch 11/100
100/100 [==============================] - 1s 8ms/step - loss: 253.3914
Epoch 12/100
100/100 [==============================] - 1s 8ms/step - los

100/100 [==============================] - 1s 8ms/step - loss: 252.6492
Epoch 98/100
100/100 [==============================] - 1s 8ms/step - loss: 252.6413
Epoch 99/100
100/100 [==============================] - 1s 8ms/step - loss: 252.6324
Epoch 100/100
100/100 [==============================] - 1s 8ms/step - loss: 252.6240


In [146]:
num_encode_layers = [1,2,3,4]
num_decode_layers = [1,2,3,4]
embedding = [16,32]
drop_frac = [0., 0.2, 0.5]
epochs = [50,100,150]
batch_size = [10,20,30]

param_grid = dict( batch_size = batch_size, epochs = epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs=-1, cv = 5)
grid_result = model.fit(np.atleast_3d(y1), np.atleast_3d(y2))

ValueError: Invalid shape for y: (1000, 100, 1)

In [147]:
grid

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x1aa2139d30>,
             iid='warn', n_jobs=-1,
             param_grid={'batch_size': [10, 20, 30], 'epochs': [50, 100, 150]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
iList = [10,100,1000,10000,100000,1000000]
x = np.linspace(0,1,100)
netWork = [8,16,32,64,128,256,512]
eCode = [1,2,3,4,5,10]
dCode = [1,2,3,4,5,10]
eBed = [16,32]
loss = 999999999
f_y1 = []
f_y2 = []
f_fit = []
information = []
for i in iList:
    y1 = []
    y2 = []
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    z1 = np.random.uniform(0,12)
    z2 = np.random.uniform(0,12)
    z3 = np.random.uniform(0,12)
    z4 = np.random.uniform(0,12)
    z5 = np.random.uniform(0,12)
    z6 = np.random.uniform(0,12)
    z7 = np.random.uniform(0,12)
    z8 = np.random.uniform(0,12)
    z9 = np.random.uniform(0,12)
    z10 = np.random.uniform(0,12)
    z11 = np.random.uniform(0,12)
    z12 = np.random.uniform(0,12)
    
    for k in range(i):
        a_ = np.random.normal(8, 1, 1)
        b_ = np.random.normal(7, 2, 1)
        c_ = np.random.normal(6, 3, 1)
        d_ = np.random.normal(5, 4, 1)
        e_ = np.random.normal(4, 5, 1)
        f_ = np.random.normal(3, 7, 1)
        y1_, y2_ = function_gen(x, a_, b_, c_, d_,e_,f_,z1,z2,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12)
        y1.append(y1_)
        y2.append(y2_)
        a.append(a_)
        b.append(b_)
        c.append(c_)
        d.append(d_)
        e.append(e_)
        f.append(f_)
        
    nor_y1 = preprocessing.normalize(y1)
    nor_y2 = preprocessing.normalize(y2)
        
    for j in netWork:
        for l in eCode:
            for m in dCode:
                for n in eBed:
                    auto_test = rnn_auto(LSTM, j, l, m, n, 100)
                    model1 = auto_test.fit(np.atleast_3d(nor_y1), np.atleast_3d(nor_y2), epochs= 200, batch_size=2000)
                    if model1.history['loss'][199] < loss:
                        loss = model1.history['loss'][199]
                        auto_test.save('auto_test')
                        nor_y1_result = auto_test.predict(np.atleast_3d(nor_y1))
                        f_y1 = nor_y1
                        f_y2 = nor_y2
                        f_fit = nor_y1_result
                        information = [i,j,l,m,n]
                    
               
            
           

In [129]:
y3 = np.asarray(y1)
y3

array([[ -974.74832884,  -968.96365556,  -963.20139351, ...,
         -511.12253491,  -507.29514552,  -503.48565476],
       [ -757.92186893,  -753.47032716,  -749.03541071, ...,
         -398.25284342,  -395.24731657,  -392.25494334],
       [-1265.73934135, -1258.2713742 , -1250.832189  , ...,
         -666.52033953,  -661.56522354,  -656.63307706],
       ...,
       [-1171.56853534, -1164.58197964, -1157.62386916, ...,
         -618.07397483,  -613.5864069 ,  -609.12186337],
       [-1078.40058881, -1072.00929965, -1065.64306967, ...,
         -567.56395097,  -563.36475678,  -559.18565001],
       [-1114.10504419, -1107.54061698, -1101.00196086, ...,
         -589.6120028 ,  -585.30314573,  -581.0149717 ]])

In [125]:
y4.shape

(1000, 100)

In [126]:
y4 = np.asarray(y2)
y4

array([[ 9.88307848,  9.89663999,  9.91021985, ..., 11.28400275,
        11.29934464, 11.31470488],
       [13.16712736, 13.17226121, 13.17745246, ..., 13.9323492 ,
        13.94305051, 13.95380922],
       [-1.09281647, -1.00171322, -0.91069384, ...,  7.35368362,
         7.43665114,  7.51953478],
       ...,
       [ 1.97558044,  2.03040755,  2.08517973, ...,  7.03806168,
         7.0875607 ,  7.13700479],
       [14.39124829, 14.40124265, 14.41128448, ..., 15.5817021 ,
        15.59630064, 15.61094665],
       [13.43125123, 13.46707389, 13.50293504, ..., 17.08526165,
        17.1248179 , 17.16441265]])

In [115]:
y5 = y4[ : , 0 : 1] 

In [116]:
y5.shape

(1000, 1)

In [134]:
y6 = y5.reshape(-1)


In [135]:
y6

array([ 9.88307848e+00,  1.31671274e+01, -1.09281647e+00,  2.23376005e+01,
        2.07947232e+01,  6.92468847e+00,  2.16828645e+01, -1.23678426e+01,
        6.08171952e+00, -3.72872134e+00,  1.36797086e+01,  2.59041811e+01,
        1.81903663e+01, -7.78974547e+00,  1.25691637e+01, -6.22730523e-01,
        2.11721207e+01,  1.43507350e+01,  1.86320777e+01,  1.29664250e+01,
        3.10908208e+00,  8.97524815e-01,  1.13538175e+01, -6.72410486e+00,
        2.48916924e+01,  9.04400283e+00,  3.82424683e+00,  2.34127721e+00,
        1.72900459e+01, -7.10684554e-01,  1.03339400e+00,  8.43162647e+00,
        9.71195983e+00,  2.34076021e+01,  1.54065523e+01,  8.78100424e+00,
       -1.20442543e+00, -2.05050362e+00, -2.73527497e+01,  1.97758017e+01,
        1.73271820e+01,  1.88597609e+01, -1.95137121e+00,  4.56546366e+00,
        1.41726612e+01,  2.79133652e+01,  8.67247252e+00,  2.12674499e+01,
       -4.56854088e+00,  1.61102822e+01,  2.62972666e+01,  1.28982241e+01,
       -8.59048259e+00,  

In [148]:
model.summary()

AttributeError: 'KerasClassifier' object has no attribute 'summary'